# Pattern Mining
In this notebook there is a small implementation in which a dataset is transformed in a transaction style dataset.

A version of apriori algorithm is also implemented and tested against an existing python implementation.

In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [2]:
X, y = load_breast_cancer(return_X_y=True, as_frame=True)

In [3]:
dataset = pd.concat((X, y), axis=1)
dataset.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [4]:
dataset["mean radius"].dtype==float

True

In [5]:
class VectorToTransactions:
    def __init__(self, n_bins=5):
        self.metadata = {}
        self.n_bins = n_bins
        self.bin_step = 1/(n_bins-2)
        
    def fit(self, X):
        for column in X:
            if X[column].dtype==float:
                metadata = {
                    "type":"numeric",
                    "bins":[],
               }
                #generate {n_bins}, including "outlier ranges" with -inf and inf
                low = -float("inf")
                for i in range(self.n_bins - 1):
                    high = np.round(
                        np.quantile( X[column], i * self.bin_step ), 3
                    )
                    window = (low, high)
                    metadata["bins"].append(window)
                    low = high
                    
                window = (low, float("inf"))
                metadata["bins"].append(window)
                self.metadata[column] = metadata
            else:
                self.metadata[column] = {
                    "type":"categorical",
                    "categories": frozenset(np.unique(X[column]))
                }

        return self
                
    def _apply_transformation(self, row):
        trans = []
        for col, value in row.items():
            if col in self.metadata:
                metadata = self.metadata[col]
                
                if metadata["type"] == "numeric":
                    
                    for bin_range in metadata["bins"]:
                        if bin_range[0]<=value<bin_range[1]:
                            trans.append(
                                f"{col}={bin_range}"
                            )
                            break
                else:
                    if value in metadata["categories"]:
                        trans.append(
                            f"{col}={int(value)}"
                        )
                    else: trans.append(f"{col}=other")
        return trans
    
    def transform(self, X):
        transitions = []
        for i in X.index:
            transitions.append(
                self._apply_transformation(X.loc[i])
            )
        return transitions

In [6]:
vec2trans = VectorToTransactions(5).fit(dataset)

In [7]:
transaction_set = vec2trans.transform(dataset)

In [8]:
from pattern_mining import Apriori
from mlxtend.frequent_patterns.apriori import apriori
from mlxtend.frequent_patterns.fpgrowth import fpgrowth
from mlxtend.preprocessing import TransactionEncoder

In [9]:
ap = Apriori(0.25).fit(transaction_set)
ap.mine(transaction_set)

Generated the frequent itemsets!


In [10]:
len(ap.freq_itemsets)

707

In [11]:
enc = TransactionEncoder().fit(transaction_set)
binary_transactions = enc.transform(transaction_set)
binary_transactions = pd.DataFrame(data=binary_transactions, columns=enc.columns_)
binary_transactions.head()

,"area error=(19.777, 34.37)","area error=(34.37, 542.2)","area error=(542.2, inf)","area error=(6.802, 19.777)","compactness error=(0.002, 0.015)","compactness error=(0.015, 0.028)","compactness error=(0.028, 0.135)","compactness error=(0.135, inf)","concave points error=(0.0, 0.009)","concave points error=(0.009, 0.013)",...,"worst smoothness=(0.071, 0.122)","worst smoothness=(0.122, 0.141)","worst smoothness=(0.141, 0.223)","worst symmetry=(0.156, 0.261)","worst symmetry=(0.261, 0.306)","worst symmetry=(0.306, 0.664)","worst texture=(12.02, 22.53)","worst texture=(22.53, 27.987)","worst texture=(27.987, 49.54)","worst texture=(49.54, inf)"
0,False,True,False,False,False,False,True,False,False,False,...,False,False,True,False,False,True,True,False,False,False
1,False,True,False,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
2,False,True,False,False,False,False,True,False,False,False,...,False,False,True,False,False,True,False,True,False,False
3,True,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,True,False,True,False,False
4,False,True,False,False,False,True,False,False,False,False,...,False,True,False,True,False,False,True,False,False,False


In [12]:
freq_items = apriori(binary_transactions, min_support=0.25)
len(freq_items)

707

In [13]:
freq_items = fpgrowth(binary_transactions, min_support=0.25)
len(freq_items)

707